# Live Demo

In [ ]:
input_sentence = nlp(input("Enter sentence: "))
fill_in_the_blank_questions = []
who_what_where_questions = []
true_true_or_false_questions= []
false_true_or_false_questions = []
multiple_choice_questions = MultipleChoice("Curie", 10, 0.20, 0.80).create()

for sentence in input_sentence.sents:
    fill_in_the_blank_questions.append(FillInTheBlank(sentence, matcher, fill_in_the_blank_pattern).create())
    who_what_where_questions.append(WhoWhatWhere(sentence, matcher, who_what_where_pattern).create())
    true_true_or_false_questions.append(TrueOrFalse(sentence, matcher, true_or_false_pattern).create())

try:
    for true_or_false_set in true_true_or_false_questions:
        for true_or_false in true_or_false_set:
            if true_or_false != None:
                false_true_or_false_questions.append(TrueOrFalseDistractor(true_or_false, 10, 0.20, 0.80).create())
except TypeError:
    false_true_or_false_questions.append(None)

print(f"Fill In The Blank Questions: {fill_in_the_blank_questions}")
print(f"Who-What-Where Questions: {who_what_where_questions}")
print(f"(True) True or False Questions: {true_true_or_false_questions}")
print(f"(False) True or False Questions: {false_true_or_false_questions}")
print(f"Multiple Choice Questions: {multiple_choice_questions}")


Enter sentence: ATP is the currency of the cell.
Fill In The Blank Questions: [[['ATP __ the currency of the cell .', ['is']], ['___ is the currency of the cell .', ['ATP']], ['ATP is the currency of the ____ .', ['cell']], ['ATP is the ________ of the cell .', ['currency']], ['___ __ the currency of the cell .', ['is', 'ATP']], ['ATP __ the currency of the ____ .', ['is', 'cell']], ['ATP __ the ________ of the cell .', ['is', 'currency']], ['___ is the currency of the ____ .', ['ATP', 'cell']], ['___ is the ________ of the cell .', ['ATP', 'currency']], ['ATP is the ________ of the ____ .', ['cell', 'currency']], ['___ __ the currency of the ____ .', ['is', 'ATP', 'cell']], ['___ __ the ________ of the cell .', ['is', 'ATP', 'currency']], ['ATP __ the ________ of the ____ .', ['is', 'cell', 'currency']], ['___ is the ________ of the ____ .', ['ATP', 'cell', 'currency']], ['___ __ the ________ of the ____ .', ['is', 'ATP', 'cell', 'currency']]]]
Who-What-Where Questions: [None]
(True) 

# Imports

In [ ]:
import spacy
from spacy.matcher import Matcher
from itertools import combinations
import numpy as np
import random
import time

# Patterns

In [ ]:
fill_in_the_blank_pattern = [[
    {"POS": "PROPN", "OP": "?"    },
    {"POS": "NOUN", "OP": "?"     },
    {"POS": "ADJECTIVE", "OP": "?"},
    {"POS": "VERB", "OP": "?"     },
    {"ENT_TYPE": "CARDINAL", "OP": "?"},
    {"ENT_TYPE": "DATE", "OP": "*"},
    {"ENT_TYPE": "EVENT", "OP": "?"},
    {"ENT_TYPE": "WORK_OF_ART", "OP": "?"},
    {"ENT_TYPE": "NORP", "OP": "?"},
    {"ENT_TYPE": "GPE", "OP": "?"},
]]

true_or_false_pattern = [[
    {"POS": "VERB", "OP": "?"},
    {"POS": "AUX", "OP": "?"},
]]

who_what_where_pattern = [[
    {"POS": "VERB", "OP": "?"},
    {"POS": "AUX", "OP": "?"},
]]

# spaCy

In [ ]:
nlp = spacy.load("en_core_web_lg")
nlp_sm = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

# Fill In The Blanks

In [ ]:
class FillInTheBlank():
    """ A class to generate fill-in-the-blanks question """

    def __init__(self, sentence, matcher, fill_in_the_blank_pattern):
        self.sentence = sentence
        self.matcher = matcher
        self.fill_in_the_blank_pattern = fill_in_the_blank_pattern
        self.matcher.add("get_to_blanks", self.fill_in_the_blank_pattern)

    """ Get all `to_blanks` token matched by the `fill_in_the_blank_pattern` """

    def _get_to_blanks(self):
        to_blanks_match = self.matcher(self.sentence)
        to_blanks = [self.sentence[start].text for _, start, end in to_blanks_match]

        return list(set(to_blanks))

    """ Get all possible `to_blanks` combination """

    def _get_to_blanks_combinations(self, to_blanks):
        to_blanks_combinations = []

        for i in range(1, len(to_blanks) + 1):
            for combo in combinations(to_blanks, i):
                to_blanks_combinations.append(combo)
        return to_blanks_combinations

    """ Structurize the question """

    def _get_blanked_sentence(self, tokens):
        blanked_sentence = []

        for sentence_token in self.sentence:
            to_append = []
            if sentence_token.text in [token for token in tokens]:
                to_append.append(len(sentence_token.text) * "_")
            else:
                to_append.append(sentence_token.text)
            blanked_sentence.append(*to_append)

        return " ".join(blanked_sentence)

    """ A one call function to generate fill-in-the-blank question """

    def create(self):
        to_blanks = self._get_to_blanks()
        combinations = self._get_to_blanks_combinations(to_blanks)

        fill_in_the_blank = []

        for combo in combinations:
            blanked_sentence = [self._get_blanked_sentence((combo))]
            blanked_sentence.append(list(combo))
            fill_in_the_blank.append(blanked_sentence)

        self.matcher.remove("get_to_blanks")

        return fill_in_the_blank

# Multiple Choice

In [ ]:
class MultipleChoice():

    """ A class to generate a multiple-choice question based on a 'who what where' question """

    def __init__(self, right_answer, similarity_token_size, minimum_similarity, maximum_similarity):
        self.right_answer = right_answer
        self.similarity_token_size = similarity_token_size
        self.minimum_similarity = minimum_similarity
        self.maximum_similarity = maximum_similarity

    """ Get the similar token from the subject of the sentence """

    def _get_distractors(self):
        try:

            most_similar_answers = nlp.vocab.vectors.most_similar(
                np.asarray([nlp.vocab.vectors[nlp.vocab.strings[self.right_answer]]]),
                n=self.similarity_token_size
            )

            similar_answers = [(nlp.vocab.strings[w]).capitalize() for w in most_similar_answers[0][0]]

            distances = most_similar_answers[2][0]

            distractors = []

            for index, distance in enumerate(distances):
                if distance >= self.minimum_similarity and distance <= self.maximum_similarity:
                    distractors.append(similar_answers[index])

            return distractors
        except (KeyError, AttributeError):
            return None
    """ Get the combination of all distractors """

    def _get_distractors_combination(self, distractors):
        return [list(combo) for combo in combinations(distractors, 3)]

    """ index 0 is always the right answer """

    def create(self):
        distractors = self._get_distractors()
        if distractors == None:
            return None

        distractors_combinations = self._get_distractors_combination(distractors)

        return distractors_combinations

# Who What Where

In [ ]:
class WhoWhatWhere():

    """ A class to generate who-what-where question """

    def __init__(self, sentence, matcher, who_what_where_pattern):
        self.sentence = sentence
        self.matcher = matcher
        self.who_what_where_pattern = who_what_where_pattern

    """ Get the `root_verb` of the input sentence """

    def _get_verbs(self):
        self.matcher.add("get_verbs", self.who_what_where_pattern)

        verbs_match = self.matcher(self.sentence)

        self.matcher.remove("get_verbs")

        verbs = [self.sentence[start] for _, start, end in verbs_match]
        return verbs

    def _get_root_verb(self, verbs):
        root_verb = None

        for verb in verbs:
            if verb.dep_ == "ROOT":
                root_verb = verb

        return root_verb

    def _get_root_verb_subjects(self, root_verb):
        root_verb_subjects = []

        for root_verb_children in root_verb.children:
            if root_verb_children.dep_ in ("nsubj", "nsubjpass"):
                root_verb_subjects.append(root_verb_children)

        return root_verb_subjects

    def _get_root_verb_objects(self, root_verb):
        root_verb_objects = []

        for root_verb_children in root_verb.children:
            if root_verb_children.dep_ in ("dobj", "dobjpass"):
                root_verb_objects.append(root_verb_children)

        return root_verb_objects


    def _get_w_question_subjects(self, root_verb_subjects):
        for root_verb_subject in root_verb_subjects:
            if root_verb_subject.ent_type_ in ("PERSON", "PRP") or root_verb_subject.tag_ in ("NN"):
                return "who"
            elif root_verb_subject.ent_type_ == "LOC":
                return "where"

        return None

    def _get_w_question_objects(self, root_verb):
        if root_verb.tag_ in ("VBD", "VBN"):
            return "what did"
        elif root_verb.tag_ in ("VBP", "VBZ"):
            return "what does"
        print(root_verb.tag_)
        return None

    def _get_remaining_text_subjects(self, root_verb_subjects):
        remaining_text = []

        for sentence_token in self.sentence:
            if sentence_token not in root_verb_subjects:
                remaining_text.append(sentence_token)

        return remaining_text

    def _get_remaining_text_objects(self, root_verb_objects):
        remaining_text = []

        for sentence_token in self.sentence:
            if sentence_token not in root_verb_objects:
                remaining_text.append(sentence_token)

        return remaining_text

    def _get_cleaned_remaining_text_subjects(self, remaining_text):
        cleaned_remaining_text = []

        for index, token in enumerate(remaining_text):
            if index == 0:
                if token.tag_ != "DT":
                    cleaned_remaining_text.append(token)
            else:
                    cleaned_remaining_text.append(token)

        cleaned = [token.text for token in cleaned_remaining_text if token.tag_ != "VBN" or token.dep_ == "ROOT"]

        return " ".join(cleaned)
    def _get_cleaned_remaining_text_objects(self, remaining_text):
        cleaned_remaining_text = []

        for index, token in enumerate(remaining_text):
            if index == len(remaining_text) - 1:
                if token.tag_ != "DT":
                    cleaned_remaining_text.append(token)
            else:
                    cleaned_remaining_text.append(token)

        cleaned = [token.text for token in cleaned_remaining_text if token.tag_ != "VBN" or token.dep_ == "ROOT"]

        return " ".join(cleaned)

    def create(self):
        who_what_where_questions = []

        verbs = self._get_verbs()
        root_verb = self._get_root_verb(verbs)

        # Who, Where
        root_verb_subjects = self._get_root_verb_subjects(root_verb)

        # What
        root_verb_objects = self._get_root_verb_objects(root_verb)


        if len(root_verb_subjects) > 0:
            w_question_subjects = self._get_w_question_subjects(root_verb_subjects)

            if w_question_subjects == None:
                return None

            remaining_text = self._get_remaining_text_subjects(root_verb_subjects)
            cleaned_remaining_text_subjects = self._get_cleaned_remaining_text_subjects(remaining_text)

            to_append = f"{w_question_subjects.capitalize()} {cleaned_remaining_text_subjects}?", [subjects.text for subjects in root_verb_subjects]
            who_what_where_questions.append(to_append)

        if len(root_verb_objects) > 0:
            w_question_objects = self._get_w_question_objects(root_verb)

            if w_question_objects == None:
                return None

            remaining_text = self._get_remaining_text_objects(root_verb_objects)
            cleaned_remaining_text_objects = self._get_cleaned_remaining_text_objects(remaining_text)

            to_append = f"{w_question_objects.capitalize()} {cleaned_remaining_text_objects}?", [objects.text for objects in root_verb_objects]
            who_what_where_questions.append(to_append)

        return who_what_where_questions

# True or False

In [ ]:
class TrueOrFalse():

    """ A class to generate true-or-false type of question """

    def __init__(self, sentence, matcher, true_or_false_pattern):
        self.sentence = sentence
        self.matcher = matcher
        self.true_or_false_pattern = true_or_false_pattern
        self.matcher.add("get_verbs", self.true_or_false_pattern)

    """ Get the verbs found in the input sentence """

    def _get_verbs(self):
        verbs_match = self.matcher(self.sentence)

        verbs = [self.sentence[start] for _, start, end in verbs_match]
        return verbs

    """ Get the root verb from the verbs """

    def _get_root_verb(self, verbs):
        root_verb = None

        for verb in verbs:
            if verb.dep_ == "ROOT":
                root_verb = verb

        return root_verb

    """ Get the helping verbs  """

    def _get_auxilliary_verbs(self, verbs):
        auxilliary_verbs = []

        for verb in verbs:
            if verb.dep_ in ("auxpass", "aux"):
                auxilliary_verbs.append(verb)

        return auxilliary_verbs

    """ Get the helping verbs  """

    def _get_main_verb(self, root_verb, auxilliary_verbs):

        if root_verb == None:
            return None

        if len(root_verb) > 0:
            return root_verb
        else:
            return auxilliary_verbs

    """ Get the type of verb (helping or not) """

    def _get_main_verb_type(self, main_verb):
        if main_verb.dep_ in ("auxpass", "aux"):
            return "AUX"
        else:
            return main_verb.pos_

    """ Verb fronting """

    def _get_verb_to_front(self, main_verb, main_verb_type):
        if main_verb_type == "AUX":
            return main_verb.text
        elif main_verb_type in ("VBD", "VBN"):
            return "did"
        elif main_verb_type in ("VBP", "VBZ"):
            return "does"
        else:
            return "Cannot determine the verb to front..."

    """ Get the subjects of the main verb """

    def _get_main_verb_subjects(self, main_verb):
        main_verb_subjects = []

        for children in main_verb.children:
            if children.dep_ in ("nsubj", "nsubjpass"):
                main_verb_subjects.append(children.text)

        return main_verb_subjects

    """ Get the remaining subject text """

    def _get_remaining_text(self, main_verb, main_verb_type, main_verb_subjects):
        remaining_text = []

        if main_verb_type == "AUX":
            for sentence_token in self.sentence:
                if sentence_token.text != main_verb.text:
                    remaining_text.append(sentence_token)
        elif main_verb_type in ("VBD", "VBN", "VBP", "VBZ"):
            for sentence_token in self.sentence:
                if sentence_token.text not in (main_verb.text, *main_verb_subjects):
                    remaining_text.append(sentence_token)
        else:
            return None

        return remaining_text

    """ Clean the remaining text """

    def _get_cleaned_remaining_text(self, remaining_text):
        cleaned_remaining_text = []

        if remaining_text == None:
            return ""

        for token in remaining_text:
            if token.tag_ not in ("-LRB-", "-RRB-", "``", "''", ".") and token.ent_type_ != "DATE" and token.dep_ != "auxpass":
                cleaned_remaining_text.append(token.text)

        return " ".join(cleaned_remaining_text)

    """ A one call function to create a true-or-false question """

    def create(self):
        true_or_false_questions = []

        verbs = self._get_verbs()
        root_verb = self._get_root_verb(verbs)
        auxilliary_verbs = self._get_auxilliary_verbs(verbs)

        main_verb = self._get_main_verb(root_verb, auxilliary_verbs)

        if main_verb == None:
            return None

        main_verb_type = self._get_main_verb_type(main_verb)

        verb_to_front = self._get_verb_to_front(main_verb, main_verb_type)

        main_verb_subjects = self._get_main_verb_subjects(main_verb)

        remaining_text = self._get_remaining_text(main_verb, main_verb_type, main_verb_subjects)
        cleaned_remaining_text = self._get_cleaned_remaining_text(remaining_text)

        # Prioritize auxilliary verbs
        if main_verb_type == "AUX":
            true_or_false_question = f"{verb_to_front.capitalize()} {cleaned_remaining_text}?"
            true_or_false_questions.append(true_or_false_question)
        elif main_verb_type in ("VBD", "VBN", "VBP", "VBZ"):
            true_or_false_question = f"{verb_to_front.capitalize()} {' '.join(main_verb_subjects)} {root_verb.lemma_} {cleaned_remaining_text}?"
            true_or_false_questions.append(true_or_false_question)
        else:
            return None

        self.matcher.remove("get_verbs")

        return true_or_false_questions

In [ ]:
class TrueOrFalseDistractor():

    """ A class to generate a distractor """

    def __init__(self, true_or_false, similar_token_size, minimum_similarity, maximum_similarity):
        self.true_or_false = true_or_false
        self.similar_token_size = similar_token_size
        self.minimum_similarity = minimum_similarity
        self.maximum_similarity = maximum_similarity

    """ Get the token to generate the distractor to """

    def _get_token_to_switch(self):
        true_or_false = nlp(self.true_or_false)

        for token in true_or_false:
            if token.dep_ in ("nsubj", "nsubjpass"):
                return token
        return ""

    """ Get the similar tokens from the tokens to switch """

    def _get_most_similar_tokens_from_token(self, token):
        try:
            most_similar_tokens = nlp.vocab.vectors.most_similar(np.asarray([nlp.vocab.vectors[nlp.vocab.strings[token.text]]]), n=self.similar_token_size)

            similar_tokens = [(nlp.vocab.strings[w]).capitalize() for w in most_similar_tokens[0][0]]

            distances = most_similar_tokens[2][0]
            tokens = []

            for index, distance in enumerate(distances):
                if distance >= self.minimum_similarity and distance <= self.maximum_similarity:
                    tokens.append(similar_tokens[index])

            return tokens
        except (KeyError, AttributeError):
            return None

    """ A one call function to create a false type question of true or false """

    def create(self):
        token_to_switch = self._get_token_to_switch()

        if token_to_switch == None:
            return None

        similar_tokens = self._get_most_similar_tokens_from_token(token_to_switch)

        if similar_tokens == None or similar_tokens == []:
            return None

        if self.true_or_false == None:
            return None


        true_or_false_sentence = self.true_or_false.split()
        true_or_false_distractor = []

        for distractor in similar_tokens:
            current_distractor = []
            for word in true_or_false_sentence:
                if token_to_switch.text == word:
                    current_distractor.append(distractor)
                else:
                    current_distractor.append(word)
            distracted_true_or_false = " ".join(current_distractor)
            true_or_false_distractor.append(distracted_true_or_false)

        return true_or_false_distractor[1:]

# Bloom's Taxonomy

In [ ]:
blooms_taxonomy = {
    "true_or_false": ["Is"],
    "remembering": [ "Choose", "Define", "Find", "How", "Label", "List", "Match", "Name", "Omit", "Recall", "Relate", "Select", "Show", "Spell", "Tell", "What", "When", "Where", "Which", "Who", "Why"],
    "understanding": ["Classify", "Compare", "Contrast", "Demonstrate", "Explain", "Extend", "Illustrate", "Infer", "Interpret", "Outline", "Relate", "Rephrase", "Show", "Summarize", "Translate"],
    "applying": ["Apply", "Build", "Choose", "Construct", "Develop", "Experiment", "Identify", "Interview", "Use", "Model", "Organize", "Plan", "Select", "Solve", "Utilize"],
    "analyzing": ["Analyze", "Assume", "Categorize", "Classify", "Compare", "Conclusion", "Contrast", "Discover", "Dissect", "Distinguish", "Divide", "Examine", "Function", "Inference", "Inspect", "List", "Motive", "Relationships", "Simplify", "Survey", "Take part in", "Test for", "Theme"],
    "evaluating": ["Agree", "Appraise", "Assess", "Award", "Choose", "Compare", "Conclude", "Criteria", "Criticize", "Decide", "Deduct", "Defend", "Determine", "Disprove", "Estimate", "Evaluate", "Explain", "Importance", "Influence", "Interpret", "Judge", "Justify", "Mark", "Measure", "Opinion", "Perceive", "Prioritize", "Prove", "Rate", "Recommend", "Rule on", "Select", "Support", "Value"],
    "creating": ["Adapt", "Build", "Change", "Choose", "Combine", "Compile", "Compose", "Construct", "Create", "Delete", "Design", "Develop", "Discuss", "Elaborate", "Estimate", "Formulate", "Happen", "Imagine", "Improve", "Invent", "Make up", "Maximize", "Minimize", "Modify", "Original", "Originate", "Plan", "Predict", "Propose", "Solution", "Solve", "Suppose", "Test", "Theory"],
}

# Assessment Generator

In [ ]:
class AssessmentGenerator():

    """ A class to find the best set of assessment questions that aligns to Bloom's Taxonomy """

    def __init__(self, assessment_questions, matcher, verb_pattern, assessment_set_size, assessment_question_size, mutation_rate, iteration_size, fitness_threshold):
        self.assessment_questions = assessment_questions
        self.matcher = matcher
        self.matcher.add("get_verbs_from_assessment_question", verb_pattern)
        self.verb_pattern = verb_pattern
        # The number of an assessment
        self.assessment_set_size = assessment_set_size
        # The number of questions in an assessment
        self.assessment_question_size = assessment_question_size
        # The chance in percent that a mutation will happen
        self.mutation_rate = mutation_rate
        # The number of iteration to stop finding for a solution
        self.iteration_size = iteration_size
        # Keep the solution if it passes this threshold
        self.fitness_threshold = fitness_threshold

    # Initialization

    def _get_random_assessment_sets(self):
        random_assessment_sets = []

        for i in range(self.assessment_set_size):
            current_assessment_set = []

            for j in range(self.assessment_question_size):
                random_assessment_question = self.assessment_questions[np.random.randint(0, len(self.assessment_questions) - 1)]

                current_assessment_set.append(random_assessment_question)

            random_assessment_sets.append(current_assessment_set)

        return random_assessment_sets

    # Evaluation

    def _get_verbs_from_assessment_question(self, assessment_question):
        doc = nlp_sm(assessment_question)
        verbs_match = self.matcher(doc)
        verbs = [doc[start].text for _, start, end in verbs_match]
        return verbs

    def _get_verb_hierarchy(self, verbs):
        for key, taxonomy_verbs in blooms_taxonomy.items():
            for taxonomy_verb in taxonomy_verbs:
                if taxonomy_verb in verbs:
                    return key

        return None

    def _get_assessment_sets_hierarchy(self, assessment_sets):
        assessment_sets_hierarchy = []

        for assessment_set in assessment_sets:
            current_assessment_set_hierarchy = []

            for assessment_question in assessment_set:
                current_assessment_question_hierarchy = {}
                current_assessment_question_verb = self._get_verbs_from_assessment_question(assessment_question)
                current_assessment_question_hierarchy[self._get_verb_hierarchy(current_assessment_question_verb)] = assessment_question
                current_assessment_set_hierarchy.append(current_assessment_question_hierarchy)
            assessment_sets_hierarchy.append(current_assessment_set_hierarchy)

        return assessment_sets_hierarchy

    # Make this configurable to the GUI (because much better to configure than manually writing codes.)
    def _get_assessment_sets_quality(self, assessment_questions_hierarchy):
        assessment_sets_quality = []

        for assessment_questions_set in assessment_questions_hierarchy:
            current_assessment_set_hierarchy = []

            for assessment_question_set in assessment_questions_set:
                for assessment_hierarchy, assessment_question in assessment_question_set.items():
                    current_assessment_set_hierarchy.append(assessment_hierarchy)

            current_assessment_set_hierarchy = set(current_assessment_set_hierarchy)


            if len(current_assessment_set_hierarchy) == 1:
                # Bad
                if set(current_assessment_set_hierarchy) == {"true_or_false"}:
                    assessment_sets_quality.append(0.25)
                elif set(current_assessment_set_hierarchy) == {None}:
                    assessment_sets_quality.append(0.35)
                else:
                    assessment_sets_quality.append(0.2)

            elif len(current_assessment_set_hierarchy) == 2:
                if set(current_assessment_set_hierarchy) == {"analyzing", "evaluating"}:
                    # Medium
                    assessment_sets_quality.append(0.4)

                # Trivia specific
                elif set(current_assessment_set_hierarchy) == {None, "true_or_false"}:
                    assessment_sets_quality.append(0.55)
                else:
                    # Bad
                    assessment_sets_quality.append(0.3)

            elif len(current_assessment_set_hierarchy) == 3:
                if set(current_assessment_set_hierarchy) == {"understanding", "analyzing", "evaluating"}:
                    # Good
                    assessment_sets_quality.append(0.7)
                elif set(current_assessment_set_hierarchy) == {None, "true_or_false", "remembering"}:
                    assessment_sets_quality.append(0.8)
                else:
                    # Medium
                    assessment_sets_quality.append(0.5)

            elif len(current_assessment_set_hierarchy) == 4:
                if set(current_assessment_set_hierarchy) == {"remembering", "understanding", "analyzing", "creating"}:
                    # Good
                    assessment_sets_quality.append(0.8)
                else:
                    # Medium
                    assessment_sets_quality.append(0.6)

            elif len(current_assessment_set_hierarchy) == 5:
                # Good
                assessment_sets_quality.append(0.9)

            elif len(current_assessment_set_hierarchy) == 6:
                # Excellent
                assessment_sets_quality.append(1.0)

        return assessment_sets_quality

    def _get_assessment_sets_fitness(self, assessment_sets_quality):
        assessment_sets_fitness = []
        for assessment_set_quality in assessment_sets_quality:
            assessment_sets_fitness.append(1/assessment_set_quality)

            # History
            history.append(1/assessment_set_quality)
        return assessment_sets_fitness

    # Selection
    def _get_assessment_sets_fitness_sum(self, assessment_sets_fitness):
        assessment_sets_fitness_sum = 0
        for assessment_set_fitness in assessment_sets_fitness:
            assessment_sets_fitness_sum += assessment_set_fitness
        return assessment_sets_fitness_sum

    def _get_assessment_sets_fitness_probabilities(self, assessment_sets_fitness_sum, assessment_sets_fitness):
        assessment_sets_fitness_probabilities = []
        previous_probability = 0

        for assessment_set_fitness in sorted(assessment_sets_fitness):
            assessment_sets_fitness_probabilities.append(previous_probability + assessment_set_fitness / assessment_sets_fitness_sum)
            previous_probability = previous_probability + assessment_set_fitness / assessment_sets_fitness_sum

        return assessment_sets_fitness_probabilities

    def _get_assessment_set_roulette(self, assessment_sets_fitness):
        assessment_sets_fitness_sum = self._get_assessment_sets_fitness_sum(assessment_sets_fitness)
        assessment_sets_fitness_probabilities = self._get_assessment_sets_fitness_probabilities(assessment_sets_fitness_sum, assessment_sets_fitness)

        wheel_result = np.random.rand()

        winner_index = 0

        for index, assessment_set_fitness_probabilities in enumerate(assessment_sets_fitness_probabilities):
            if wheel_result < assessment_set_fitness_probabilities:
                winner_index = index
                break

        return winner_index

    # Crossover
    def _get_assessment_sets_crossover(self, assessment_set_roulette_1, assessment_set_roulette_2):
        random_median = np.random.randint(1, len(assessment_set_roulette_1))
        if np.random.randint(0, 1) == 0:
            assessment_questions_1 = assessment_set_roulette_1[:random_median]
            assessment_questions_2 = assessment_set_roulette_2[random_median:]
            return assessment_questions_1 + assessment_questions_2
        else:
            assessment_questions_1 = assessment_set_roulette_2[:random_median]
            assessment_questions_2 = assessment_set_roulette_1[random_median:]
            return assessment_questions_1 + assessment_questions_2
    # Mutation
    def _get_assessment_sets_mutation(self, assessment_sets_crossover):
        mutation = np.random.randint(1, 101)

        if mutation <= self.mutation_rate:
            random_assessment_position = np.random.randint(0, len(assessment_sets_crossover) - 1)
            random_assessment_question = np.random.randint(0, len(self.assessment_questions) - 1)
            assessment_sets_crossover[random_assessment_position] = self.assessment_questions[random_assessment_question]
            return assessment_sets_crossover

        return assessment_sets_crossover

    def create(self):
        solutions = self._get_random_assessment_sets()

        solution_sets = []

        for solution in range(self.iteration_size):
            assessment_sets_hierarchy = self._get_assessment_sets_hierarchy(solutions)
            assessment_sets_quality = self._get_assessment_sets_quality(assessment_sets_hierarchy)
            assessment_sets_fitness = self._get_assessment_sets_fitness(assessment_sets_quality)

            new_solutions = []

            print(f"Solution {solution}")

            for new_solution in range(self.assessment_set_size):
                roulette_1_winner = self._get_assessment_set_roulette(assessment_sets_fitness)
                roulette_2_winner = self._get_assessment_set_roulette(assessment_sets_fitness)

                roulette_attempts = 0
                while roulette_1_winner == roulette_2_winner and roulette_attempts < 100:
                    roulette_2_winner = self._get_assessment_set_roulette(assessment_sets_fitness)
                    roulette_attempts += 1

                if roulette_1_winner == roulette_2_winner:
                    assessment_set_roulette_1 = solutions[roulette_1_winner]
                    assessment_set_roulette_2 = solutions[roulette_2_winner]
                else:
                    assessment_set_roulette_1 = solutions[roulette_1_winner]
                    assessment_set_roulette_2 = solutions[roulette_2_winner]

                # Crossover
                assessment_sets_crossover = self._get_assessment_sets_crossover(assessment_set_roulette_1, assessment_set_roulette_2)
                # Mutation
                assessment_sets_mutation = self._get_assessment_sets_mutation(assessment_sets_crossover)

                new_solutions.append(assessment_sets_mutation)


            for index, solution in enumerate(solutions):
                current_fitness = assessment_sets_fitness[index]
                if current_fitness <= self.fitness_threshold:
                    solution_sets.append(solution)

            solutions = new_solutions

        self.matcher.remove("get_verbs_from_assessment_question")

        for solution in solution_sets:
            if len(set(solution)) < self.assessment_question_size:
                solution_sets.remove(solution)

        return solution_sets

# Dependencies

In [ ]:
!python3 -m spacy download en_core_web_sm
!python3 -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 84.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 1.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
